In [7]:
import os
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [8]:
merged = pd.read_csv("merged_with_gender.tsv", delimiter="\t")
# filter out examples where we failed to infer gender
genders = ['male', 'female']
df_filtered = merged[merged['gender'].isin(genders)]
df_filtered.shape

(53401, 11)

How many females v males?

In [9]:
df_filtered["gender"].describe()

count     53401
unique        2
top        male
freq      38272
Name: gender, dtype: object

So about 72% male

In [41]:
vectorizer = CountVectorizer(max_features=20000, min_df=5, ngram_range=(1, 2), binary=True)
X = vectorizer.fit_transform(df_filtered["review-text"])
y = df_filtered["gender"]

Note; should probably pull out and pool reviews for physicians -- here we treat individual reviews for the same physician as multiple / independent instances

In [42]:
param_grid = {"C":[.01, .1, 1, 10, 100]}
clf = GridSearchCV(LogisticRegression(), param_grid=param_grid, scoring="f1_macro")

In [43]:
clf.fit(X,y)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.01, 0.1, 1, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='f1_macro', verbose=0)

In [44]:
clf.best_score_

0.92289133147666569

In [45]:
def show_most_informative_features(vectorizer, clf, n=50):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [47]:
show_most_informative_features(vectorizer, clf.best_estimator_, n=100)

	-2.3243	she            		2.5000	he             
	-1.9253	her            		1.7128	his            
	-0.6667	she is         		1.4590	him            
	-0.4463	her staff      		0.6649	he is          
	-0.3860	recommend her  		0.4184	surgery        
	-0.3342	commentsshe    		0.4008	his staff      
	-0.2895	her office     		0.3343	man            
	-0.2886	and her        		0.3343	recommend him  
	-0.2843	her patients   		0.3161	and his        
	-0.2538	she has        		0.3059	guy            
	-0.2377	herself        		0.2895	to him         
	-0.2009	doctor she     		0.2655	mother         
	-0.1984	see her        		0.2487	he has         
	-0.1896	to her         		0.2480	his patients   
	-0.1837	her to         		0.2474	he was         
	-0.1807	she did        		0.2427	him to         
	-0.1666	she was        		0.2324	pain           
	-0.1549	with her       		0.2266	daughter       
	-0.1529	my son         		0.2164	his office     
	-0.1512	her for        		0.2133	commentshe     
	-0.1449	practice   